In [31]:
%pip install kubeflow
%pip install -U kubeflow-training


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
%pip show kubeflow-training

Name: kubeflow-training
Version: 1.9.1
Summary: Training Operator Python SDK
Home-page: https://github.com/kubeflow/training-operator/tree/master/sdk/python
Author: Kubeflow Authors
Author-email: hejinchi@cn.ibm.com
License: Apache License Version 2.0
Location: /opt/app-root/lib64/python3.11/site-packages
Requires: certifi, kubernetes, retrying, setuptools, six, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from kubernetes import client
from kubeflow.training import TrainingClient

api_server = ""
token = ""
namespace="test-namespace"

# Configure the API client with the server and token
configuration = client.Configuration()
configuration.host = api_server
configuration.api_key = {"authorization": f"Bearer {token}"}
configuration.verify_ssl = False  # Disable SSL verification

print("successfully authenticated!")

successfully authenticated!


In [37]:
client = TrainingClient()

In [ ]:
# create configmap to store driver_stats_training.jsonl files
import os
from kubernetes import client, config
from kubernetes.client.rest import ApiException

# Define ConfigMap name
CONFIGMAP_NAME = "fms-hf-tuning-config"

# Define headers for API authentication
configuration = client.Configuration()
configuration.host = api_server
configuration.verify_ssl = False  # Set to True if using a valid CA certificate
configuration.api_key = {"authorization": f"Bearer {token}"}

# Load Kubernetes client configuration
client.Configuration.set_default(configuration)
api_instance = client.CoreV1Api()

# Read training dataset file from the current directory
CONFIG_FILE = "./config_lora.json"
DATASET_FILE = "./twitter_complaints_small.json"

if os.path.exists(DATASET_FILE):
    with open(DATASET_FILE, "r", encoding="utf-8") as file:
        dataset_content = file.read()
else:
    raise FileNotFoundError(f"Dataset file '{DATASET_FILE}' not found in current directory.")

if os.path.exists(CONFIG_FILE):
    with open(CONFIG_FILE, "r", encoding="utf-8") as file:
        config_content = file.read()
else:
    raise FileNotFoundError(f"Dataset file '{CONFIG_FILE}' not found in current directory.")

# Define ConfigMap data
if dataset_content:
    configmap_data = {
        f"twitter_complaints.json": dataset_content,
        f"config_lora.json": config_content
    }

    # Create the ConfigMap object
    configmap = client.V1ConfigMap(
        metadata=client.V1ObjectMeta(name=CONFIGMAP_NAME),
        data=configmap_data
    )
    
    # Apply ConfigMap to Kubernetes
    try:
        api_instance.create_namespaced_config_map(namespace=namespace, body=configmap)
        print(f"ConfigMap '{CONFIGMAP_NAME}' created successfully in namespace '{namespace}'.")
    except ApiException as e:
        if e.status == 409:
            print(f"ConfigMap '{CONFIGMAP_NAME}' already exists. Updating it...")
            api_instance.replace_namespaced_config_map(name=CONFIGMAP_NAME, namespace=namespace, body=configmap)
        else:
            print(f"Error creating ConfigMap: {e}")

/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.abdhumal-osd2.mbh6.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ConfigMap 'fms-hf-tuning-config' created successfully in namespace 'test-namespace'.


In [ ]:
from kfto_sdk_mnist import train_func
from kubeflow.training.models import V1Volume, V1VolumeMount, V1PersistentVolumeClaimVolumeSource

# Start PyTorchJob with 2 Workers and 2 GPU per Worker (e.g. multi-node, multi-worker job).
client.create_job(
    job_kind="PyTorchJob",
    name="sft",
    base_image="quay.io/modh/fms-hf-tuning:v2.7.1",
    num_workers=2,
    resources_per_worker={"gpu": "1"},
    env_vars={
        # HuggingFace
        "HF_HOME": "/mnt/output/.cache",
        "HF_TOKEN": "",
        "SFT_TRAINER_CONFIG_JSON_PATH": "/mnt/data/config_lora.json",
        # NCCL / RCCL
        "NCCL_DEBUG": "INFO",
    },
    volume_mounts=[
        {
           "name": "data-volume",
           "mountPath": "/mnt/data"  # Directory where training dataset files will be available
        },
        V1VolumeMount(name="shared", mount_path="/mnt/output"),
    ],
    volumes=[
       {
           "name": "data-volume",
           "configMap": {"name": "fms-hf-tuning-config"}  # Reference ConfigMap consisting training dataset file
       },
        V1Volume(name="shared",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="shared"))
        
    ]
)

In [39]:
import time
print("waiting.")
while not client.is_job_succeeded(name="sft"):
    print(".", end="")
    time.sleep(1)
print("PytorchJob succeeded!")

waiting.
PytorchJob succeeded!


In [40]:
client.delete_job("sft")